Fonctions à faire : 
- animation de graphe
- k-vicinity
- donner l'algo de MDRW (ou autre) et faire faire la fonction

Interpréter les résultats des figures
Créer les figures (plot)

Aller + loin : créer une version distribuée de MDRW+RNS

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from swarm_sim import *

### Chargement des données
Rappeler la forme des données et la signification des champs

A faire : charger la totalité. Modifier le PATH si nécessaire.

In [5]:
PATH = '..\data\Traces_Nanosatellites\\track_0.csv'

df = pd.read_csv(PATH, sep=',', header=0)
df['coords'] = ['x','y','z']
df = df.set_index('coords', drop=True)

df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
coords,,,,,,,,,,,,,,,,,,,,,
x,-4.850747e+05,-4.937755e+05,-5.024536e+05,-5.111086e+05,-5.197401e+05,-5.283476e+05,-5.369309e+05,-5.454894e+05,-5.540229e+05,-5.625310e+05,...,1.490967e+06,1.494274e+06,1.497573e+06,1.500865e+06,1.504149e+06,1.507425e+06,1.510693e+06,1.513954e+06,1.517206e+06,1.520451e+06
y,-7.066530e+05,-7.201117e+05,-7.335372e+05,-7.469290e+05,-7.602864e+05,-7.736088e+05,-7.868957e+05,-8.001464e+05,-8.133603e+05,-8.265368e+05,...,2.218138e+06,2.223313e+06,2.228477e+06,2.233629e+06,2.238769e+06,2.243898e+06,2.249015e+06,2.254120e+06,2.259214e+06,2.264296e+06
z,-2.026363e+06,-2.019625e+06,-2.012795e+06,-2.005871e+06,-1.998855e+06,-1.991747e+06,-1.984548e+06,-1.977257e+06,-1.969876e+06,-1.962403e+06,...,3.692429e+06,3.686602e+06,3.680756e+06,3.674890e+06,3.669006e+06,3.663103e+06,3.657180e+06,3.651239e+06,3.645278e+06,3.639299e+06


In [ ]:
# A faire

"""
NB : le chargement peut être assez long (plusieurs minutes en fonction de la machine).
Vous pouvez visualiser la progression du chargement avec la fonction tqdm() qui fonctionne comme suit :

with tqdm(total = nb_iterations_max, desc = description) as pbar:
    for i in nb_iterations_max:
        # do something
        pbar.update(1)
"""

Travailler avec une portée de connexion de 30 km

In [6]:
CONNECTION_RANGE = 30000

Convertir les données en objets Swarm (lire la doc pour le formattage).

Afin de faciliter la suite du traitement, pensez à créer un objet Swarm par pas de temps et les stocker dans un dictionnaire tel que :

dict_swarm[timestamp] = Swarm

In [ ]:
help(Swarm.__init__)

In [7]:
# A faire

""" 
NB : le formattage peut également prendre un peu de temps en fonction de la quantité de données à formatter.
N'hésitez pas à utiliser la fonction tqdm() décrite plus haut si vous le souhaitez.
"""

Afficher le contenu du Swarm ainsi que la description du noeud de votre choix.

In [ ]:
# A faire

### Visualisation de la topologie
Afficher l'essaim à un pas de temps donné, puis à un autre (vous trouverez peut-être la fonction dans le module). 

Qu'observez-vous au niveau de la topologie ?

In [12]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

%matplotlib inline 

In [17]:
# A faire

Vous avez sûrement remarqué que la fonction plot_edges() ne fonctionne pas. Pourquoi ?

In [16]:
# Réponse

Etablissez les connexions entre noeuds voisins grâce à la fonction neighbor_matrix() (plus d'informations sur Swarm.distance_matrix), puis affichez l'état de l'essaim grâce à la fonction plot_edges().

In [18]:
# A faire

A partir des fonctions disponibles dans le module, créez une fonction plot_trajectory() qui affiche la trajectoire de l'essaim autour de la Lune.

Notez que la Lune se situe au point (0,0,0) dans ce système de coordonnées, mais il n'est pas nécessaire de l'afficher.


In [13]:
# A faire

En vous basant sur les coordonnées des noeuds, calculer la période de révolution d'un satellite.


In [14]:
# A faire

Exercice bonus : animer l'évolution de la topologie de l'essaim sur une période de révolution complète.

In [15]:
from matplotlib.animation import FuncAnimation

In [19]:
# A faire

### 3. Caractérisation du réseau
Nous allons maintenant déterminer les caractéristiques du réseau en calculant certaines métriques. La métrique la plus simple est le degré, soit le nombre de voisins directs d'un noeud.

Calculer le degré moyen de l'essaim à l'instant 0, puis calculer l'évolution de ce degré moyen sur une révolution complète. Vérifiez que vous avez bien établi la connexion sur une révolution complète avant !

In [ ]:
help(Swarm.degree)

In [20]:
# A faire